# Whole Pipeline

Now that we have separately addressed the different modules of the pipeline, let's put them all together in a single pipe.

1. Load the image;
2. Find the liver regions;
3. Find structures in Sox9 staining (ducts, vessels, Sox9+ cells, not well stained areas);
4. Find portal regions.

After this, we should look into generating tables with the information describing each region.

In [1]:
from pathlib import Path

import pyclesperanto_prototype as cle
import napari
import numpy as np

from gulliver.io import get_image, save_img
from gulliver.segmenter import (
    find_structures,
    segment_liver,
    clean_segmentations,
)

cle.select_device("RTX")
print(cle.get_device().name)

c:\Users\agucor\Anaconda3\envs\bile_duct_clf\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


NVIDIA GeForce RTX 3060 Laptop GPU


In [2]:
DATA_DIR = Path("../data/143 E8.czi")

image = get_image(DATA_DIR, 1)

Once the image is loaded, we should first look for liver structures.

In [3]:
segmentations = find_structures(
    image["Sox9"]["image"], chunk_shape=(5 * 1024, 5 * 7024)
)

In [4]:
liver = segmentations.create_group("liver")
liver.create_dataset(
    "labels",
    data=segment_liver(image["DAPI"]["image"]),
)

<zarr.core.Array '/liver/labels' (27746, 13050) int32>

In [5]:
clean_segmentations(segmentations)

In [6]:
save_img("test.zar", image, segmentations)